In [1]:
import numpy as np
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 399839 files belonging to 7 classes.
Found 50947 files belonging to 7 classes.


In [5]:
base_model = keras.applications.EfficientNetB4(input_shape=(224,224,3), include_top = False, weights='imagenet')

In [6]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [7]:
for layer in base_model.layers:
    layer.trainable = False

In [8]:
op = keras.optimizers.SGD(momentum=0.9, nesterov=True)
model.compile(optimizer=op,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(training_s, validation_data=validation_s, epochs=5)

Epoch 1/5
12495/12495 [==============================] - 2018s 161ms/step - loss: 1.4169 - accuracy: 0.4557 - val_loss: 1.3359 - val_accuracy: 0.4991
Epoch 2/5
12495/12495 [==============================] - 1985s 159ms/step - loss: 1.3436 - accuracy: 0.4875 - val_loss: 1.2881 - val_accuracy: 0.5154
Epoch 3/5
12495/12495 [==============================] - 1997s 160ms/step - loss: 1.3237 - accuracy: 0.4962 - val_loss: 1.2657 - val_accuracy: 0.5193
Epoch 4/5
12495/12495 [==============================] - 1985s 159ms/step - loss: 1.3137 - accuracy: 0.5008 - val_loss: 1.2591 - val_accuracy: 0.5262
Epoch 5/5
12495/12495 [==============================] - 2000s 160ms/step - loss: 1.3069 - accuracy: 0.5039 - val_loss: 1.2639 - val_accuracy: 0.5318


In [10]:
for layer in base_model.layers:
    layer.trainable = True

In [11]:
opt = keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9, nesterov=True)

In [12]:
es = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [13]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_s, validation_data=validation_s, epochs=10, callbacks=[es])

Epoch 1/10
12495/12495 [==============================] - 8978s 717ms/step - loss: 1.2696 - accuracy: 0.5332 - val_loss: 0.9659 - val_accuracy: 0.6408
Epoch 2/10
12495/12495 [==============================] - 8945s 716ms/step - loss: 0.9557 - accuracy: 0.6409 - val_loss: 0.8689 - val_accuracy: 0.6757
Epoch 3/10
12495/12495 [==============================] - 8909s 713ms/step - loss: 0.8810 - accuracy: 0.6696 - val_loss: 0.8178 - val_accuracy: 0.6934
Epoch 4/10
12495/12495 [==============================] - 8974s 718ms/step - loss: 0.8334 - accuracy: 0.6881 - val_loss: 0.7836 - val_accuracy: 0.7071
Epoch 5/10
12495/12495 [==============================] - 8922s 714ms/step - loss: 0.7969 - accuracy: 0.7019 - val_loss: 0.7558 - val_accuracy: 0.7185
Epoch 6/10
12495/12495 [==============================] - 9002s 720ms/step - loss: 0.7674 - accuracy: 0.7130 - val_loss: 0.7334 - val_accuracy: 0.7287
Epoch 7/10
12495/12495 [==============================] - 8943s 716ms/step - loss: 0.7407 - ac

In [14]:
model.evaluate(training_s)

12495/12495 [==============================] - 1750s 140ms/step - loss: 0.5897 - accuracy: 0.7830


[0.5896645784378052, 0.783017635345459]

In [15]:
model.evaluate(validation_s)

1593/1593 [==============================] - 226s 142ms/step - loss: 0.6643 - accuracy: 0.7599


[0.6642904281616211, 0.7599465847015381]

In [16]:
model.save("0924_B4_nesterov.h5")